In [5]:
!pip install --upgrade pip


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [7]:
!pip install tensorflow
!pip install keras
!pip install gradio
!pip install pandas
!pip install scikit-learn


  Using cached gradio-5.21.0-py3-none-any.whl.metadata (16 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 114.7 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
import gradio as gr
import pandas as pd
from sklearn.model_selection import train_test_split


In [9]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 70 bytes


In [10]:
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset

Dataset URL: https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset
License(s): CC-BY-NC-SA-4.0
 93% 38.0M/41.0M [00:02<00:00, 30.9MB/s]
100% 41.0M/41.0M [00:02<00:00, 19.5MB/s]


In [11]:
from zipfile import ZipFile

file_name = "/content/fake-and-real-news-dataset.zip"

with ZipFile(file_name, 'r') as zip:
    zip.extractall()
    print('Done')


Done


In [12]:
from zipfile import ZipFile

# Specify the path to your zip file
file_name = "/content/fake-and-real-news-dataset.zip"

# Using ZipFile to extract the contents of the zip file
with ZipFile(file_name, 'r') as zip:
    zip.extractall()  # This will extract all the files in the same directory as the zip file
    print('Extraction complete!')


Extraction complete!


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
import gradio as gr

# Load the datasets
fake_news = pd.read_csv('/content/Fake.csv')
true_news = pd.read_csv('/content/True.csv')

# Assign labels
fake_news['label'] = 0  # 0 for fake
true_news['label'] = 1  # 1 for true

# Combine the datasets
data = pd.concat([fake_news, true_news], ignore_index=True)

# Shuffle the data
data = data.sample(frac=1).reset_index(drop=True)

# Combine title and text for better context
data['text'] = data['title'] + " " + data['text']
X = data['text'].values
y = data['label'].values

# Encode labels to one-hot
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y)

# Tokenize text
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(sequences, maxlen=500)


In [15]:
model = Sequential([
    Embedding(5000, 128, input_length=500),
    SpatialDropout1D(0.2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(2, activation='softmax')  # Two outputs for fake and real
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d (SpatialDropout1D) │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/5
562/562 ━━━━━━━━━━━━━━━━━━━━ 814s 1s/step - accuracy: 0.9074 - loss: 0.2256 - val_accuracy: 0.9801 - val_loss: 0.0642
Epoch 2/5
562/562 ━━━━━━━━━━━━━━━━━━━━ 864s 1s/step - accuracy: 0.9808 - loss: 0.0568 - val_accuracy: 0.9817 - val_loss: 0.0564
Epoch 3/5
562/562 ━━━━━━━━━━━━━━━━━━━━ 800s 1s/step - accuracy: 0.9864 - loss: 0.0432 - val_accuracy: 0.9830 - val_loss: 0.0565
Epoch 4/5
562/562 ━━━━━━━━━━━━━━━━━━━━ 791s 1s/step - accuracy: 0.9857 - loss: 0.0443 - val_accuracy: 0.9876 - val_loss: 0.0479
Epoch 5/5
562/562 ━━━━━━━━━━━━━━━━━━━━ 805s 1s/step - accuracy: 0.9784 - loss: 0.0614 - val_accuracy: 0.9899 - val_loss: 0.0341


In [28]:
import gradio as gr

def predict_news(text):
    # Tokenize and pad the input text
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=500)
    # Predict and return probabilities
    pred = model.predict(padded)
    # Decide the label based on the probabilities
    fake_prob = float(pred[0][0]) * 100
    real_prob = float(pred[0][1]) * 100
    if fake_prob > real_prob:
        result = f"Fake: {fake_prob:.2f}% 🔴"
    else:
        result = f"Real: {real_prob:.2f}% ✅"
    return result

# Remove the 'layout' argument, or upgrade Gradio
iface = gr.Interface(
    fn=predict_news,
    inputs=gr.Textbox(lines=4, placeholder="Enter News Text Here...", label="Input News Text"),
    outputs=gr.Label(),
    title="Fake News Detection",
    description="Enter a piece of news text below to predict whether it is Fake or Real. A red cross (🔴) indicates a higher probability of being fake, and a green check (✅) indicates a higher probability of being real.",
    theme="huggingface",  # Apply a modern theme for better visual appeal
    css='''
        body { font-family: Arial, sans-serif; }
        .gr-textbox { border: 2px solid #3572b0; border-radius: 5px; }
        .gr-label { font-size: 16px; font-weight: bold; color: #28527a; }
        .gr-interface { background-color: #f8f9fa; padding: 20px; border-radius: 8px; }
        .gr-output { background-color: #e9ecef; border-radius: 5px; padding: 10px; }
        .gr-button, .gr-button:hover { background-color: #007bff; border-color: #007bff; }  /* Blue buttons */
    ''',  # Custom CSS for additional styling
    # layout="vertical"  # Remove or comment out this line
)

iface.launch()

/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1102: UserWarning: Cannot load huggingface. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/huggingface (Request ID: Root=1-67d906f9-10548e5b01b1c81944a2fee8;c240ca62-44f2-48a5-95f4-d8fb626a61ca)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1671f5fc8c3c25d230.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
